In [1]:
import sys
sys.path.insert(0, r'../../quetzal/')
import syspy

import pandas as pd
ipath = r'inputs/'

# FRATAR algorithm

Algorithme itératif pour modifier une matrice OD via ses émissions / attractions

In [2]:
from syspy import distribution

In [3]:
ods = pd.read_csv(ipath + r'ods.csv', index_col=0)
ods.head()

,origin,destination,periode,volume
0,zone_0,zone_0,HPM,2.551684
3,zone_0,zone_1,HPM,1.176192
5,zone_0,zone_10,HPM,0.495365
10,zone_0,zone_100,HPM,8.010701
14,zone_0,zone_101,HPM,27.553181


ATTENTION: la matrice doit être complète

In [4]:
import numpy as np
zone_ids = list(set(ods['origin'].values).union(set(ods['destination'].values)))
all_ods_df = pd.DataFrame(
    index=[zone_ids, zone_ids],
    data=np.zeros(len(zone_ids))
).unstack().fillna(0).stack()

all_ods_df.index.names = ['origin', 'destination']

In [5]:
ods = all_ods_df.join(
    ods.set_index(['origin', 'destination'])['volume']
).fillna(0)[['volume']]
ods.head()

volume
origin destination           
zone_0 zone_0        2.551684
       zone_1        1.176192
       zone_10       0.495365
       zone_100      8.010701
       zone_101     27.553181

On calcule les émissions et attractions actuelles pour les modifier

In [6]:
productions = ods.groupby('origin')['volume'].sum()
attractions = ods.groupby('destination')['volume'].sum()

In [7]:
new_productions = productions.copy()
new_productions.loc['zone_352'] *= 10

In [8]:
trips_array = ods.unstack().values
prod_array = new_productions.sort_index().values
attr_array = attractions.sort_index().values

In [9]:
from syspy.distribution import distribution
distribution.CalcFratar??

Signature:
distribution.CalcFratar(
    ProdA,
    AttrA,
    Trips1,
    maxIter=10,
    print_balance=False,
)
Source:   
def CalcFratar(ProdA, AttrA, Trips1, maxIter=10, print_balance=False):
    '''Calculates fratar trip distribution
       ProdA = Production target as array
       AttrA = Attraction target as array
       Trips1 = Seed trip table for fratar
       maxIter (optional) = maximum iterations, default is 10
       Returns fratared trip table
    '''
    # print('Checking production, attraction balancing:')
    sumP = sum(ProdA)
    sumA = sum(AttrA)
    # print('Production: ', sumP)
    # print('Attraction: ', sumA)
    if sumP != sumA:
        if print_balance:
            print('Productions and attractions do not balance, attractions will be scaled to productions!')
        AttrA = AttrA * (sumP / sumA)
    else:
        if print_balance:
            print('Production, attraction balancing OK.')
    # Run 2D balancing --->
    for balIter in range(0, maxIter):
       

In [10]:
# Fratar
new_trips = distribution.CalcFratar(
    prod_array, attr_array, trips_array, 10, True
)

Productions and attractions do not balance, attractions will be scaled to productions!


In [11]:
new_ods = pd.DataFrame(
    data=new_trips,
    index=ods.unstack().index,
    columns=ods.unstack().columns
).stack().reset_index()

In [12]:
new_ods.groupby('origin')['volume'].sum().round(-1).equals(productions.round(-1))

False

In [13]:
new_ods.groupby('origin')['volume'].sum().round(-1).equals(new_productions.round(-1))

True

In [14]:
new_ods.groupby('destination')['volume'].sum().round(-1).equals(attractions.round(-1))

False

In [15]:
new_ods.groupby('destination')['volume'].sum().round(-1).equals(
    (attractions * new_productions.sum() / productions.sum()).round(-1)
)

True